# FRP Data Processing and Analysis Notebook

This notebook processes CVS. files, filters, and cleans the data, and provides basic visualization. Follow the steps and instructions provided.

## Import Necessary Libraries

First, we need to import the necessary libraries.


In [7]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, Polygon
from pyresample import geometry
import numpy as np
import contextily as ctx
from pyresample.geometry import AreaDefinition
from pyresample import kd_tree
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='pyproj')


## Define Helper Functions

These functions will help in filtering, cleaning, and processing the data.

In [8]:
# Import custom data processing functions
from data_processing import filter_and_clean_data, process_files, regrid_and_reproject_data, combine_data, plot_reprojected_data

## Usage Instructions

## Define Parameters and Directories

Specify the parameters and directories needed for the data processing.


In [9]:
# File path to the CSV file
file_path = 'DL_FIRE_June_2020/fire_archive_SV-C2_478543.csv'

# Load your point data from the CSV file
df = pd.read_csv(file_path)

df

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
0,56.053772,160.645752,367.00,0.40,0.44,2020-06-01,106,N,VIIRS,h,1,327.56,271.11,D,1
1,68.647507,152.401840,340.27,0.43,0.38,2020-06-01,112,N,VIIRS,n,1,292.12,6.01,D,0
2,68.716637,152.415970,337.17,0.43,0.38,2020-06-01,112,N,VIIRS,n,1,287.83,3.63,D,0
3,68.649506,152.411041,334.82,0.43,0.38,2020-06-01,112,N,VIIRS,n,1,288.88,6.01,D,0
4,68.644478,152.406845,337.82,0.43,0.38,2020-06-01,112,N,VIIRS,n,1,289.47,3.14,D,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371846,27.041206,49.563564,316.85,0.47,0.47,2020-06-30,2248,N,VIIRS,n,1,305.80,3.85,N,2
371847,27.102461,49.309978,324.75,0.45,0.47,2020-06-30,2248,N,VIIRS,n,1,299.07,1.91,N,2
371848,27.106693,49.311085,316.58,0.45,0.47,2020-06-30,2248,N,VIIRS,n,1,299.44,1.91,N,2
371849,29.549994,28.530140,311.10,0.66,0.73,2020-06-30,2248,N,VIIRS,n,1,291.78,2.41,N,0


In [10]:
# Filter data
filtered_df = df[(df['type'] == 0) & (df['confidence'] == 'h') & (df['daynight'] == 'D')]

# Select relevant columns
df_selected = filtered_df[['latitude', 'longitude', 'acq_date', 'frp']]

# Group by 'acq_date'
filtered_data = df_selected.groupby('acq_date')

## Define Area Definition

From pyresample https://pyresample.readthedocs.io/en/latest/howtos/geo_def.html#areadefinition

In [11]:
area_id = 'arctic_circle'
description = 'Arctic Circle region grid'
proj_id = 'north_polar_stereographic'
projection = '+proj=stere +lat_0=90 +lat_ts=70 +lon_0=-45 +k=1 +datum=WGS84 +units=m +no_defs'
width = 100 # number of pixels
height = 100 # number of pixels
area_extent = (-4511076.79, -4511076.79, 4511076.79, 4511076.79)

area_def = AreaDefinition(area_id, description, proj_id, projection, width, height, area_extent)

## Step 4: Reproject and Plot Data

Reproject the data and plot the results.

Methods:

'nearest': Uses the nearest neighbor interpolation to resample the data.

'average': Averages the data values within the defined grid cells.

'count': Counts the number of non-NaN observations within each grid cell.

'sum': Sums the data values within each grid cell.

'min': Finds the minimum data value within each grid cell.

'max': Finds the maximum data value within each grid cell.

'median': Computes the median data value within each grid cell.

'abs_max': Finds the absolute maximum value within each grid cell.

Pyresample kd.tree: https://pyresample.readthedocs.io/en/latest/howtos/swath.html#pyresample-kd-tree

& bucket: https://pyresample.readthedocs.io/en/latest/howtos/swath.html#pyresample-bucket

In [17]:
import pandas as pd
import dask.array as da
from pyresample import geometry, kd_tree
from pyresample.geometry import AreaDefinition

# Filter data based on your conditions
df = pd.read_csv('DL_FIRE_June_2020/fire_archive_SV-C2_478543.csv') 
filtered_df = df[(df['type'] == 0) & (df['confidence'] == 'h') & (df['daynight'] == 'D')]

# Select relevant columns
df_selected = filtered_df[['latitude', 'longitude', 'acq_date', 'frp']]

# Regrid and reproject function
def regrid_and_reproject_data(lats, lons, data, area_def, method='nearest'):
    swath_def = geometry.SwathDefinition(lons=lons, lats=lats)

    if method == 'nearest':
        reprojected_data = kd_tree.resample_nearest(
            swath_def, data, area_def, radius_of_influence=50000, epsilon=0.5, fill_value=None)
    else:
        # Convert lats, lons, and data to Dask arrays with chunking
        lons = da.from_array(lons, chunks=(lons.shape[0]//10, lons.shape[1]//10))
        lats = da.from_array(lats, chunks=(lats.shape[0]//10, lats.shape[1]//10))
        data = da.from_array(data, chunks=(data.shape[0]//10, data.shape[1]//10))

        # Apply nearest method for simplicity (you can expand to other methods)
        reprojected_data = kd_tree.resample_nearest(
            swath_def, data, area_def, radius_of_influence=50000, epsilon=0.5, fill_value=None)

    lons_new, lats_new = area_def.get_lonlats()
    return reprojected_data, lons_new, lats_new

# Group by 'acq_date' and process each group
reprojected_results = {}
for acq_date, group in df_selected.groupby('acq_date'):
    lats = group['latitude'].to_numpy()
    lons = group['longitude'].to_numpy()
    data = group['frp'].to_numpy()  # Using FRP as the data for regridding

    # Regrid and reproject the data for each acquisition date
    reprojected_data, lons_new, lats_new = regrid_and_reproject_data(lats, lons, data, area_def, method='nearest')

    # Store the result in the dictionary with acq_date as the key
    reprojected_results[acq_date] = {
        'reprojected_data': reprojected_data,
        'lons_new': lons_new,
        'lats_new': lats_new
    }

In [31]:
import matplotlib.pyplot as plt
from datetime import datetime
import cartopy.crs as ccrs

# Function to plot reprojected data
def plot_reprojected_data(reprojected_datasets, area_def, plot_date=None, cmap='RdYlBu_r', vmin=0, vmax=30):
    """
    Plots the reprojected data for a specific date.

    Parameters:
    - reprojected_datasets: dict, reprojected datasets with each date as key
    - area_def: AreaDefinition, area definition for the target grid (not used here directly)
    - plot_date: str, date to plot (format: YYYY-MM-DD)
    - cmap: str, colormap for the plot
    - vmin: float, minimum value for colormap
    - vmax: float, maximum value for colormap
    """
    if plot_date:
        # Convert the plot_date string to a datetime.date object
        try:
            plot_date = datetime.strptime(plot_date, '%Y-%m-%d').date()
        except ValueError:
            print(f"Invalid date format: {plot_date}. Please use YYYY-MM-DD format.")
            return
        
        # If a specific date is provided, plot data for that date
        if plot_date in reprojected_datasets:
            data = reprojected_datasets[plot_date]
            
            # Plot setup
            fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'projection': ccrs.NorthPolarStereo()})
            
            # Set the map extent and add coastlines
            ax.set_extent([-180, 180, 55, 90], crs=ccrs.PlateCarree())
            ax.coastlines()
            
            # Add gridlines with labels on the left and bottom
            gl = ax.gridlines(draw_labels=True)
            gl.top_labels = False
            gl.right_labels = False
            
            # Plot the data using pcolormesh
            mesh = ax.pcolormesh(
                data["lons_new"], 
                data["lats_new"], 
                data["reprojected_data"], 
                cmap=cmap, 
                vmin=vmin, 
                vmax=vmax, 
                transform=ccrs.PlateCarree()
            )
            
            # Add a color bar to show data values
            plt.colorbar(mesh, ax=ax, orientation='vertical', label='Reprojected Data Values')
            
            # Add a title
            plt.title(f'Reprojected Data on {plot_date}')
            plt.show()
        else:
            print(f"No data available for the specified date: {plot_date}")
    else:
        # If no date is provided, inform the user
        print("No date specified for plotting. Please provide a date to plot the data.")

# Example usage:
# Assuming `reprojected_results` is your dictionary containing the reprojected data
plot_reprojected_data(reprojected_results, area_def, plot_date='2020-06-01')



No data available for the specified date: 2020-06-01


# Reproject the data
reprojected_datasets = {}
for date, data in filtered_data.items():
    reprojected_data, lons_new, lats_new = regrid_and_reproject_data(
        data["latitude"], data["longitude"], data[variable_name], area_def, method='average')
    
    reprojected_datasets[date] = {
        'lons_new': lons_new,
        'lats_new': lats_new,
        'reprojected_data': reprojected_data
    }

## Step 5: Plot Data for a Specific Date

Provide the date, colormap, and value range for which you want to plot the reprojected data.

In [15]:
# Define the date, colormap, and value range you want to plot
date_to_plot = '2020-06-20'  # Change this to the desired date
colormap = 'Reds'  # Change this to the desired colormap
vmin = 1  # Change this to the desired minimum value
vmax = 4  # Change this to the desired maximum value

# Call the plotting function with the specified parameters
plot_reprojected_data(reprojected_datasets, area_def, plot_date=date_to_plot, cmap=colormap, vmin=vmin, vmax=vmax)


No data available for the specified date: 2020-06-20


In [20]:
# Plot daily averages
plot_daily_averages(reprojected_datasets)

NameError: name 'plot_daily_averages' is not defined

In [ ]:
# After generating reprojected_datasets_aerosol
import pickle

with open('reprojected_datasets_aai.pkl', 'wb') as f:
    pickle.dump(reprojected_datasets, f)

In [ ]:

# Load aerosol index data
with open('reprojected_datasets_aai.pkl', 'rb') as f:
    reprojected_datasets_aerosol = pickle.load(f)

# Load carbon monoxide data
with open('reprojected_datasets_co.pkl', 'rb') as f:
    reprojected_datasets_co = pickle.load(f)


import numpy as np
from netCDF4 import Dataset

def save_to_netcdf(reprojected_datasets_aerosol, reprojected_datasets_co, area_def, file_name):
    with Dataset(file_name, 'w', format='NETCDF4') as ncfile:
        # Create dimensions
        sample_date = next(iter(reprojected_datasets_aerosol))
        sample_data = reprojected_datasets_aerosol[sample_date]
        lat_dim = ncfile.createDimension('lat', sample_data['lats_new'].shape[0])
        lon_dim = ncfile.createDimension('lon', sample_data['lons_new'].shape[1])
        x_dim = ncfile.createDimension('x', area_def.width)
        y_dim = ncfile.createDimension('y', area_def.height)
        
        # Create variables for coordinates
        lats_var = ncfile.createVariable('latitude', np.float32, ('y', 'x'))
        lons_var = ncfile.createVariable('longitude', np.float32, ('y', 'x'))
        x_var = ncfile.createVariable('x', np.float32, ('x',))
        y_var = ncfile.createVariable('y', np.float32, ('y',))
        
        # Write latitude and longitude data
        lats_var[:, :] = sample_data['lats_new']
        lons_var[:, :] = sample_data['lons_new']
        
        # Write x and y coordinate data
        x_var[:] = np.linspace(area_def.area_extent[0], area_def.area_extent[2], area_def.width)
        y_var[:] = np.linspace(area_def.area_extent[1], area_def.area_extent[3], area_def.height)
        
        # Add attributes to coordinates
        lats_var.units = 'degrees_north'
        lons_var.units = 'degrees_east'
        x_var.units = 'meters'
        y_var.units = 'meters'
        
        # Create CRS variable
        crs = ncfile.createVariable('crs', 'i4')
        crs.spatial_ref = '+proj=stere +lat_0=90 +lat_ts=70 +lon_0=-45 +datum=WGS84 +units=m +no_defs'
        crs.grid_mapping_name = 'polar_stereographic'
        crs.straight_vertical_longitude_from_pole = -45.0
        crs.latitude_of_projection_origin = 90.0
        crs.standard_parallel = 70.0
        crs.false_easting = 0.0
        crs.false_northing = 0.0
        
        # Add aerosol index data
        for date, data in reprojected_datasets_aerosol.items():
            data_var = ncfile.createVariable(f'aerosol_index_{date}', np.float32, ('y', 'x',), zlib=True)
            data_var[:, :] = data['reprojected_data']
            data_var.units = 'aerosol_index'
            data_var.coordinates = 'latitude longitude'
            data_var.grid_mapping = 'crs'
        
        # Add carbon monoxide data
        for date, data in reprojected_datasets_co.items():
            data_var = ncfile.createVariable(f'carbon_monoxide_{date}', np.float32, ('y', 'x',), zlib=True)
            data_var[:, :] = data['reprojected_data']
            data_var.units = 'mol/m^2'
            data_var.coordinates = 'latitude longitude'
            data_var.grid_mapping = 'crs'

# Save to NetCDF
save_to_netcdf(reprojected_datasets_aerosol, reprojected_datasets_co, area_def, 'combined_output_1.nc')
